## Печатнов Юрий М05-895б

Домашнее задание по теории решеток

В данной работе взят датасет Tic-Tac-Toe

Разработана собственная модификация алгоритма классификации на основе генераторов

И сравнена с `RandomForestClassifier` и `GradientBoostingClassifier` из `sklearn.ensemble`

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from IPython.display import display, clear_output

In [2]:
import sklearn.ensemble

Для начала бинаризуем признаки, чтобы сделать задачу более подходящей для применения fca

Затем на одних и тех же данных применим три алгоритма классификации

In [7]:
def preprocess(df):
    df = df.copy()
    last_col = list(df)[-1]
    for col in list(df)[:-1]:
        df[col + "_0"] = (df[col] == 'x') | (df[col] == 'o')
        df[col + "_1"] = (df[col] == 'o')
        df = df.drop([col], axis=1)
    df["y"] = (df[last_col] == 'positive')
    df = df.drop([last_col], axis=1)
    return df

def get_common_features(X):
    if len(X) == 0:
        return None, None
    else:
        mask = (X == X[0]).sum(axis=0) == len(X)
        return mask, X[0][mask]

Рассматриваю две симетричные задачи и считаю `score` для каждой. Если `score` больше для положительной задачи то ответ положительный. Иначе отрицательный.
`score` считаю так: при фиксированном тестовом запросе перебираю тренировочные положительные примеры, считаю пересечения с ними. Затем делаю замыкание каждого пересечения пользуясь отрицательными примерами. Если окажется так, что замыкание лежит в тестовом запросе, то в `score` прибавляю (`размер замыкания` - `размер пересечения`) * `поддержка`.

Так же фильтрую пересечения по размеру, поддержке и достоверности.

In [8]:
class FCAPredictor:
    def __init__(self, min_support=0.001, min_confidence=0.001, min_intersect=3):
        for name in self.__init__.__code__.co_varnames:
            if name != 'self':
                setattr(self, name, locals()[name])
        
    def fit(self, X, y):
        X = np.array(X)  # L x F
        y = (np.array(y) != 0)  # L
        
        self.examples = [X[~y], X[y]]  # negative and positive examples 
        self.examples_X = X
        self.examples_y = y
        
        self.targets_sums = [(~y).sum(), y.sum()]
        
    def predict_one(self, x):
        x = np.array(x)  # F
        
        # masks of equal features
        intersections = (self.examples_X == x)
        
        results = []
        
        # score
        rating2 = [0, 0]
        rating = [0, 0]
        for elem, cand, t in zip(self.examples_X, intersections, self.examples_y):
            min_subset_cardinality = cand.sum()
            if min_subset_cardinality < self.min_intersect:
                continue
            including_mask = ((intersections & cand).sum(axis=1) >= min_subset_cardinality)
            
            corresponding_targets = (self.examples_y[including_mask] == t)
            corresponding_X = self.examples_X[including_mask]
            
            support = corresponding_targets.sum() / self.targets_sums[int(t)]
            confidence = (~corresponding_targets).sum() / self.targets_sums[int(~t)]
    
            common_features_mask, common_features_values = \
                get_common_features(corresponding_X[~corresponding_targets])
            
            if common_features_mask is not None and np.all(x[common_features_mask] == common_features_values) \
                and support > self.min_support and confidence > self.min_confidence:
                rating2[int(~t)] += (common_features_mask.sum() - min_subset_cardinality) * support
             
            #rating[int(t)] += (support > self.min_support) & (unconfidence < self.max_unconfidence)
            #rating[int(t)] += ((support > self.min_support) & (support > 3 * unconfidence)) * intersection_cardinality
            rating[int(t)] += support > 2 * confidence
        
        rating2[1] /= self.targets_sums[1]
        rating2[0] /= self.targets_sums[0]
        return rating2[1] >= rating2[0]
            
        rating[1] /= self.targets_sums[1]
        rating[0] /= self.targets_sums[0]
        return rating[1] >= rating[0]
        
    def predict(self, X):
        X = np.array(X)  # L' x F
        y = np.array([self.predict_one(x) for x in X], dtype=np.bool)  # L
        return y        

Подсчет метрик и запуски

In [9]:
def calc_metrics(y_pred, y_real):
    metrics = dict(
        TP=(y_pred & y_real).sum(),
        FP=(y_pred & ~y_real).sum(),
        TN=(~y_pred & ~y_real).sum(),
        FN=(~y_pred & y_real).sum(),
        accuracy = (y_pred == y_real).sum() / len(y_pred),
    )
    metrics.update(
        precision=metrics["TP"] / (metrics["TP"] + metrics["FP"]),
        recall=metrics["TP"] / (metrics["TP"] + metrics["FN"]),
    )
    return metrics


def try_it(train, test, predictor=None):
    train = preprocess(train)
    test = preprocess(test)
    predictor = predictor if predictor is not None else FCAPredictor()
    predictor.fit(train.drop(["y"], axis=1), train["y"])
    predictions = predictor.predict(test.drop(["y"], axis=1))

    
    real_targets = np.array(test["y"])
    return calc_metrics(predictions, real_targets)

def show_on_set(i, predictor=None):
    predictor = predictor if predictor is not None else FCAPredictor(min_intersect=12)
    return try_it(pd.read_csv("../../dataset/train%d.csv" % i), pd.read_csv("../../dataset/test%d.csv" % i), predictor)

In [10]:
rows = []
for i in range(1, 11):
    pref = "%d_" % i
    row = {
        pref + "FCA": show_on_set(i, FCAPredictor(min_intersect=12)), 
        pref + "RANDOMFOREST": show_on_set(i, sklearn.ensemble.RandomForestClassifier(n_estimators=200, max_depth=10)),
        pref + "GRADIENT-BOOSTING": show_on_set(i, sklearn.ensemble.GradientBoostingClassifier(n_estimators=200, max_depth=10))
    }
    display(row)
    for k, v in row.items():
        v["method"] = k.split('_')[1]
        v["data_set"] = k.split('_')[0]
    rows += row.values()
clear_output(wait=True)
df = pd.DataFrame(rows)
display(df)
display(df.groupby(['method'], as_index=False).mean())

,FN,FP,TN,TP,accuracy,data_set,method,precision,recall
0,0,0,32,61,1.000000,1,FCA,1.000000,1.000000
1,5,1,31,56,0.935484,1,RANDOMFOREST,0.982456,0.918033
2,6,0,32,55,0.935484,1,GRADIENT-BOOSTING,1.000000,0.901639
3,0,5,31,51,0.942529,2,FCA,0.910714,1.000000
4,0,5,31,51,0.942529,2,RANDOMFOREST,0.910714,1.000000
5,2,4,32,49,0.931034,2,GRADIENT-BOOSTING,0.924528,0.960784
6,0,2,33,65,0.980000,3,FCA,0.970149,1.000000
7,0,2,33,65,0.980000,3,RANDOMFOREST,0.970149,1.000000
8,1,1,34,64,0.980000,3,GRADIENT-BOOSTING,0.984615,0.984615
9,0,3,27,59,0.966292,4,FCA,0.951613,1.000000


,method,FN,FP,TN,TP,accuracy,precision,recall
0,FCA,0.6,2.0,31.2,62.0,0.972582,0.968643,0.990484
1,GRADIENT-BOOSTING,2.5,1.8,31.4,60.1,0.954668,0.971585,0.959690
2,RANDOMFOREST,1.1,2.4,30.8,61.5,0.962764,0.961933,0.982369


Видим, что на задаче адаптированной под FCA, этот метод выдает лучшие результаты.
Но нужно учитывать адаптированность и то, что для FCA подбирались лучшие параметры, а для других моделей - нет.

Далее код для подбора параметров

In [46]:
log_file = "log_txt"
with open(log_file, 'w') as f:
    f.write("\n")
for min_support in [0.001, 0.003, 0.005]:
    for min_confidence in [0.001, 0.003, 0.005]:
        for min_intersect in [12]:
            params = {
                "min_support": min_support,
                "min_confidence": min_confidence,
                "min_intersect": min_intersect,
            }
            predictor = FCAPredictor(**params)
            params.update(
                try_it(pd.read_csv("../../dataset/train2.csv"), pd.read_csv("../../dataset/test2.csv"), predictor)
            )
            with open(log_file, 'a') as f:
                f.write(str(params) + "\n")
            print(params)

{'min_support': 0.001, 'min_confidence': 0.001, 'min_intersect': 12, 'TP': 51, 'FP': 5, 'TN': 31, 'FN': 0, 'accuracy': 0.94252873563218387, 'precision': 0.9107142857142857, 'recall': 1.0}
{'min_support': 0.001, 'min_confidence': 0.003, 'min_intersect': 12, 'TP': 51, 'FP': 5, 'TN': 31, 'FN': 0, 'accuracy': 0.94252873563218387, 'precision': 0.9107142857142857, 'recall': 1.0}
{'min_support': 0.001, 'min_confidence': 0.005, 'min_intersect': 12, 'TP': 49, 'FP': 3, 'TN': 33, 'FN': 2, 'accuracy': 0.94252873563218387, 'precision': 0.94230769230769229, 'recall': 0.96078431372549022}
{'min_support': 0.003, 'min_confidence': 0.001, 'min_intersect': 12, 'TP': 51, 'FP': 5, 'TN': 31, 'FN': 0, 'accuracy': 0.94252873563218387, 'precision': 0.9107142857142857, 'recall': 1.0}
{'min_support': 0.003, 'min_confidence': 0.003, 'min_intersect': 12, 'TP': 51, 'FP': 5, 'TN': 31, 'FN': 0, 'accuracy': 0.94252873563218387, 'precision': 0.9107142857142857, 'recall': 1.0}
{'min_support': 0.003, 'min_confidence': 0.

KeyboardInterrupt: 

## TODO

Далее идею можно развивать: например стоит сделать подбор идеальных параметров частью процесса обучения:

разделить учебную выборку на пре-учебную и пре-тестовую,

далее перебирать параметры, обучаться на пре-учебной, валидироваться на пре-тестовой,

и так выбрать лучшие параметры.

В итоговой модели использовать полученные параметры и всю учебную выборку.

In [11]:
!jupyter nbconvert lattices.ipynb --to markdown

[NbConvertApp] Converting notebook lattices.ipynb to markdown
[NbConvertApp] Writing 14931 bytes to lattices.md


In [4]:
!head ../test1.csv


V1,V2,V3,V4,V5,V6,V7,V8,V9,V10
x,x,x,x,o,o,o,x,o,positive
x,x,x,x,o,b,o,b,o,positive
x,x,x,o,o,x,o,x,o,positive
x,x,x,o,o,b,x,o,b,positive
x,x,x,b,o,b,o,o,x,positive
x,x,x,b,b,o,b,o,b,positive
x,x,o,o,x,o,o,x,x,positive
x,x,o,o,x,b,o,x,b,positive
x,x,b,x,o,o,x,o,b,positive
